# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [14]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import json

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [15]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [16]:
from api import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [17]:
# #pk_e5bbd27b3ec143fba02eeb209be04273
# symbol='AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url).json()
# data


# symbol = 'TSLA'
# IEX_API_TOKEN = 'pk_e5bbd27b3ec143fba02eeb209be04273'
# url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_API_TOKEN}'
# r = requests.get(url, verify = False)
# data = r.text
# data


# symbol = 'AAPL'
# api_url = f'https://ALGORITHMIC.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# print(api_url)
# data = requests.get(api_url).json()
# data

symbol = 'AAPL'
IEX_API_TOKEN = 'pk_e5bbd27b3ec143fba02eeb209be04273'
api_url = f'https://ALGORITHMIC.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

# symbol = 'TSLA'
# IEX_API_TOKEN = 'pk_e5bbd27b3ec143fba02eeb209be04273'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_API_TOKEN}'
# data = requests.get(api_url).json()
# data


  # This line will print the content of the 'data' variable


[{'avgTotalVolume': 52141290, 'calculationPrice': 'tops', 'change': 1.155, 'changePercent': 0.00608, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': 'IEX real time price', 'highTime': 1701450705139, 'iexAskPrice': 191.12, 'iexAskSize': 250, 'iexBidPrice': 191.1, 'iexBidSize': 400, 'iexClose': 191.105, 'iexCloseTime': 1701451159249, 'iexLastUpdated': 1701451159249, 'iexMarketPercent': 0.03306345353475085, 'iexOpen': 190.305, 'iexOpenTime': 1701441000722, 'iexRealtimePrice': 191.105, 'iexRealtimeSize': 100, 'iexVolume': 629392, 'lastTradeTime': 1701451159249, 'latestPrice': 191.105, 'latestSource': 'IEX real time price', 'latestTime': '12:19:19 PM', 'latestUpdate': 1701451159249, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, '

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

Usage:
rename ['latestPrice'] -> Desired Metric

In [18]:
first_entry = data[0]
latest_price = first_entry['latestPrice']
market_cap = first_entry['marketCap']

print('Latest Price:', latest_price)
print('Market Cap:', market_cap)


Latest Price: 191.105
Market Cap: 2972208670960


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [19]:
# my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
# final_dataframe = pd.DataFrame(columns = my_columns)
# final_dataframe
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']

# Create an empty DataFrame with the specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

# Apply styling to center-align the columns
styled_dataframe = final_dataframe.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
])

# Display the styled DataFrame
styled_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [20]:
# final_dataframe = final_dataframe.append(
#                                         pd.Series(['AAPL', 
#                                                    data['latestPrice'], 
#                                                    data['marketCap'], 
#                                                    'N/A'], 
#                                                   index = my_columns), 
#                                         ignore_index = True)
# final_dataframe
if isinstance(data, list) and len(data) > 0:
    data = data[0]

# Now 'data' is guaranteed to be a dictionary

# Assuming final_dataframe is a pandas DataFrame
final_dataframe = pd.DataFrame(columns=my_columns)

# Extract relevant information from 'data' and append to the DataFrame
final_dataframe = final_dataframe.append(
    pd.Series([
        symbol,
        '${:.2f}'.format(data.get('latestPrice', 0)),  # Format Price as currency
        '${:,.0f}'.format(data.get('marketCap', 0)),   # Format Market Cap as currency with commas
        'N/A'
    ], index=my_columns),
    ignore_index=True
)

# Apply styling to center-align the columns
styled_dataframe = final_dataframe.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
])

# Display the styled DataFrame
styled_dataframe

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/3947115113.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,$191.10,"$2,972,208,670,960",N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [29]:
import pandas as pd
import requests



# Define the list of S&P 500 stocks or load it from a file/database
# Example: Load from a CSV file containing a column named 'Ticker'
# s&p500_tickers = pd.read_csv('sp500_tickers.csv')['Ticker'].tolist()

# For illustration purposes, using a small set of sample tickers
snp500_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Define column names for the DataFrame
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']

# Create an empty DataFrame with the specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

# Iterate over each stock ticker
for symbol in snp500_tickers:
    IEX_API_TOKEN = 'pk_e5bbd27b3ec143fba02eeb209be04273'
    api_url = f'https://ALGORITHMIC.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()

    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        data = response.json()

        # Print the formatted response content for better readability
        print(f"Response Content for {symbol}: {json.dumps(data, indent=2)}")

        # Access 'latestPrice' and 'marketCap' from the first element of the list
        first_entry = data[0]
        latest_price = first_entry.get('latestPrice')
        market_cap = first_entry.get('marketCap')

        if latest_price is not None and market_cap is not None:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol, latest_price, market_cap, 'N/A'], index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data for {symbol} is incomplete or missing 'latestPrice' and 'marketCap' keys.")
    except requests.exceptions.RequestException as err:
        print(f"Error: {err}")
        print(f"Response Content: {response.text}")  # Print the response content for debugging

# Display or further use the final_dataframe
print(final_dataframe)



Response Content for AAPL: [
  {
    "avgTotalVolume": 52141290,
    "calculationPrice": "tops",
    "change": 1.35,
    "changePercent": 0.00711,
    "close": null,
    "closeSource": "official",
    "closeTime": null,
    "companyName": "Apple Inc",
    "currency": "USD",
    "delayedPrice": null,
    "delayedPriceTime": null,
    "extendedChange": null,
    "extendedChangePercent": null,
    "extendedPrice": null,
    "extendedPriceTime": null,
    "high": null,
    "highSource": null,
    "highTime": null,
    "iexAskPrice": 191.33,
    "iexAskSize": 100,
    "iexBidPrice": 191.3,
    "iexBidSize": 400,
    "iexClose": 191.3,
    "iexCloseTime": 1701451642890,
    "iexLastUpdated": 1701451642890,
    "iexMarketPercent": 0.032918257873418864,
    "iexOpen": 190.305,
    "iexOpenTime": 1701441000722,
    "iexRealtimePrice": 191.3,
    "iexRealtimeSize": 100,
    "iexVolume": 643238,
    "lastTradeTime": 1701451642890,
    "latestPrice": 191.3,
    "latestSource": "IEX real time price

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/4012314543.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


Response Content for MSFT: [
  {
    "avgTotalVolume": 26800327,
    "calculationPrice": "tops",
    "change": -5.81,
    "changePercent": -0.01533,
    "close": null,
    "closeSource": "official",
    "closeTime": null,
    "companyName": "Microsoft Corporation",
    "currency": "USD",
    "delayedPrice": null,
    "delayedPriceTime": null,
    "extendedChange": null,
    "extendedChangePercent": null,
    "extendedPrice": null,
    "extendedPriceTime": null,
    "high": null,
    "highSource": null,
    "highTime": null,
    "iexAskPrice": 373.1,
    "iexAskSize": 300,
    "iexBidPrice": 373.08,
    "iexBidSize": 100,
    "iexClose": 373.1,
    "iexCloseTime": 1701451642897,
    "iexLastUpdated": 1701451642897,
    "iexMarketPercent": 0.03128925544170517,
    "iexOpen": 376.895,
    "iexOpenTime": 1701441000302,
    "iexRealtimePrice": 373.1,
    "iexRealtimeSize": 100,
    "iexVolume": 477095,
    "lastTradeTime": 1701451642897,
    "latestPrice": 373.1,
    "latestSource": "IEX re

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/4012314543.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


Response Content for GOOGL: [
  {
    "avgTotalVolume": 25262719,
    "calculationPrice": "tops",
    "change": -0.71,
    "changePercent": -0.00536,
    "close": null,
    "closeSource": "official",
    "closeTime": null,
    "companyName": "Alphabet Inc - Class A",
    "currency": "USD",
    "delayedPrice": null,
    "delayedPriceTime": null,
    "extendedChange": null,
    "extendedChangePercent": null,
    "extendedPrice": null,
    "extendedPriceTime": null,
    "high": null,
    "highSource": null,
    "highTime": null,
    "iexAskPrice": 131.84,
    "iexAskSize": 200,
    "iexBidPrice": 131.82,
    "iexBidSize": 100,
    "iexClose": 131.82,
    "iexCloseTime": 1701451637056,
    "iexLastUpdated": 1701451637056,
    "iexMarketPercent": 0.051475726397857184,
    "iexOpen": 131.85,
    "iexOpenTime": 1701441000053,
    "iexRealtimePrice": 131.82,
    "iexRealtimeSize": 100,
    "iexVolume": 738777,
    "lastTradeTime": 1701451637056,
    "latestPrice": 131.82,
    "latestSource": "

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/4012314543.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


Response Content for AMZN: [
  {
    "avgTotalVolume": 48233619,
    "calculationPrice": "tops",
    "change": 0.74,
    "changePercent": 0.00507,
    "close": null,
    "closeSource": "official",
    "closeTime": null,
    "companyName": "Amazon.com Inc.",
    "currency": "USD",
    "delayedPrice": null,
    "delayedPriceTime": null,
    "extendedChange": null,
    "extendedChangePercent": null,
    "extendedPrice": null,
    "extendedPriceTime": null,
    "high": null,
    "highSource": null,
    "highTime": null,
    "iexAskPrice": 146.75,
    "iexAskSize": 100,
    "iexBidPrice": 146.72,
    "iexBidSize": 100,
    "iexClose": 146.77,
    "iexCloseTime": 1701451645964,
    "iexLastUpdated": 1701451645964,
    "iexMarketPercent": 0.023851889993380764,
    "iexOpen": 146,
    "iexOpenTime": 1701441000306,
    "iexRealtimePrice": 146.77,
    "iexRealtimeSize": 100,
    "iexVolume": 424591,
    "lastTradeTime": 1701451645964,
    "latestPrice": 146.77,
    "latestSource": "IEX real time

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/4012314543.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


Response Content for TSLA: [
  {
    "avgTotalVolume": 126517446,
    "calculationPrice": "tops",
    "change": -1.16,
    "changePercent": -0.00483,
    "close": null,
    "closeSource": "official",
    "closeTime": null,
    "companyName": "Tesla Inc",
    "currency": "USD",
    "delayedPrice": null,
    "delayedPriceTime": null,
    "extendedChange": null,
    "extendedChangePercent": null,
    "extendedPrice": null,
    "extendedPriceTime": null,
    "high": null,
    "highSource": null,
    "highTime": null,
    "iexAskPrice": 246,
    "iexAskSize": 400,
    "iexBidPrice": 238.93,
    "iexBidSize": 100,
    "iexClose": 238.92,
    "iexCloseTime": 1701451653523,
    "iexLastUpdated": 1701451653523,
    "iexMarketPercent": 0.0049093366960475445,
    "iexOpen": 233.04,
    "iexOpenTime": 1701441000336,
    "iexRealtimePrice": 238.92,
    "iexRealtimeSize": 50,
    "iexVolume": 377889,
    "lastTradeTime": 1701451653523,
    "latestPrice": 238.92,
    "latestSource": "IEX real time pr

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/4012314543.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


In [30]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,191.30,2975241457600,N/A
1,MSFT,373.10,2772977074950,N/A
2,GOOGL,131.82,1662118380000,N/A
3,AMZN,146.77,1516725669107,N/A
4,TSLA,238.92,748648663151,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [31]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [65]:
import pandas as pd
import requests

# Replace 'YOUR_IEX_CLOUD_API_TOKEN' with your actual IEX Cloud API token

# Define your column names
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']

# Your stock symbols (replace this with your actual list of symbols)
stocks = pd.DataFrame({'Ticker': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']})

# Split symbols into chunks of 100 (adjust as needed)
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [','.join(symbol_group) for symbol_group in symbol_groups]

# Create an empty DataFrame with the specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

# Iterate over each symbol string
for symbol_string in symbol_strings:
    # Construct the API URL using the sandbox endpoint
    IEX_CLOUD_API_TOKEN = 'pk_e5bbd27b3ec143fba02eeb209be04273'

    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'

    try:
        # Make the API request
        response = requests.get(batch_api_call_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)

        # Print the response content for debugging
        print(f"Response Content: {response.text}")

        # Parse the JSON content
        data = response.json()

        # Check if the response data is not empty
        if data:
            # Iterate over each symbol in the current string
            for symbol in symbol_string.split(','):
                # Check if the symbol is present in the response data
                if symbol in data:
                    # Extract data for the symbol and append to the DataFrame
                    final_dataframe = final_dataframe.append(
                        pd.Series([symbol, 
                                   data[symbol]['quote']['latestPrice'], 
                                   data[symbol]['quote']['marketCap'], 
                                   'N/A'], 
                                  index=my_columns), 
                        ignore_index=True
                    )
                else:
                    print(f"Data for {symbol} not found in the response.")
        else:
            print("Empty response from the API.")
    except requests.exceptions.RequestException as err:
        print(f"Error making API request: {err}")

# Display or further use the final_dataframe
print(final_dataframe)


Response Content: {"AAPL":{"quote":{"avgTotalVolume":52141290,"calculationPrice":"tops","change":1.42,"changePercent":0.00748,"close":null,"closeSource":"official","closeTime":null,"companyName":"Apple Inc","currency":"USD","delayedPrice":null,"delayedPriceTime":null,"extendedChange":null,"extendedChangePercent":null,"extendedPrice":null,"extendedPriceTime":null,"high":null,"highSource":"IEX real time price","highTime":1701453602740,"iexAskPrice":191.4,"iexAskSize":200,"iexBidPrice":191.38,"iexBidSize":100,"iexClose":191.37,"iexCloseTime":1701454138592,"iexLastUpdated":1701454138592,"iexMarketPercent":0.03308440900367077,"iexOpen":190.305,"iexOpenTime":1701441000722,"iexRealtimePrice":191.37,"iexRealtimeSize":100,"iexVolume":726696,"lastTradeTime":1701454138592,"latestPrice":191.37,"latestSource":"IEX real time price","latestTime":"1:08:58 PM","latestUpdate":1701454138592,"latestVolume":null,"low":null,"lowSource":null,"lowTime":null,"marketCap":2976330150240,"oddLotDelayedPrice":null,

/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/3609161345.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/3609161345.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/3609161345.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/6r/ty6k941x2rzczjrqyxc8s40h0000gn/T/ipykernel_31024/3609161345.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

## Strategic Portfolio Allocation for Optimal Capital Deployment: A Financial Analysis

In our pursuit of financial excellence, we've implemented a methodical approach to allocate capital effectively across a portfolio of stocks. The following steps elucidate the financial strategy:

1. User Input for Portfolio Valuation:

Commencing our financial strategy, we engage the user, typically a discerning portfolio manager or investor, to input the total valuation of their investment portfolio. This critical input serves as the foundation for our capital allocation strategy.
python
Copy code
portfolio_size = input("Enter the value of your portfolio: ")
2. Validation of Numerical Input:

Recognizing the significance of precise numerical inputs, our code includes a validation mechanism to ensure that the user provides a valid numeric value. In the event of an invalid input, the user is prompted to re-enter the value.
python
Copy code
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio: ")
3. Calculation of Position Size:

Employing a prudent financial approach, we calculate the position size by dividing the total portfolio value equally among the selected stocks. This strategic calculation sets the stage for the precise allocation of capital.
python
Copy code
position_size = val / len(final_dataframe)
4. Determination of Optimal Share Quantities:

Our code systematically traverses each stock in the portfolio, taking into account its prevailing market price. Leveraging the calculated position size and the current stock price, it strategically determines the optimal number of whole shares to be acquired. This approach optimally utilizes the allocated budget.
python


Conclusively, the code displays the augmented final_dataframe, now enriched with the meticulously determined number of shares recommended for acquisition for each stock. This process aligns with our commitment to prudent capital management and risk mitigation in the dynamic financial landscape.
python
Copy code
print(final_dataframe)
This meticulous financial strategy ensures an optimized and diversified investment approach, harmonizing with our overarching objective of maximizing returns while prudently managing financial risks.

We'll do that next.

In [66]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio: 5


In [68]:
import math

# Assuming you already have 'final_dataframe' DataFrame

# Get the portfolio size from the user
while True:
    try:
        portfolio_size = float(input("Enter the value of your portfolio: "))
        if portfolio_size >= 0:
            break
        else:
            print("Portfolio value should be a non-negative number. Try again.")
    except ValueError:
        print("That's not a valid number! Try again.")

# Calculate position size for each stock
position_size = portfolio_size / len(final_dataframe.index)

# Calculate the number of shares to buy for each stock
for i in range(len(final_dataframe)):
    price = final_dataframe.loc[i, 'Price']
    shares_to_buy = math.floor(position_size / price)
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = shares_to_buy

# Display the updated final_dataframe
print(final_dataframe)


Enter the value of your portfolio:  10000


  Ticker   Price Market Capitalization Number Of Shares to Buy
0   AAPL  191.37         2976330150240                      10
1   MSFT  374.27         2781672821875                       5
2  GOOGL  131.71         1660731390000                      15
3   AMZN  146.95         1518585794613                      13
4   TSLA  238.75          748115973244                       8


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [69]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [70]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [73]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
final_dataframe.to_excel('recommended_trades.xlsx', index=False)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [74]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [75]:
final_dataframe.to_excel('recommended_trades.xlsx', index=False)
